In [2]:
import gzip
import pickle
import random

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline

In [3]:
#compute probabilities

In [14]:
mendelian_errors = pickle.load(gzip.open('mendelian_errors.pickle.gz', 'rb'))
feature_fit = np.load(gzip.open('feature_fit.npy.gz', 'rb'))

OSError: Failed to interpret file <gzip _io.BufferedReader name='feature_fit.npy.gz' 0x7f7c3602e630> as a pickle

In [8]:
len(mendelian_errors), len(list(filter(lambda x: x[0] > 0,mendelian_errors.values())))

(10905732, 541688)

In [11]:
def get_stats():
    error_mq0 = []
    ok_mq0 = []
    for i in acceptable_position_to_genotype():
        if i == 60000:
            break
        my_mendelian_errors = mendelian_errors.get(POS[i], (0, 0))
        if my_mendelian_errors[0] > 0:
            error_mq0.append(MQ0[i])
        else:
            ok_mq0.append(MQ0[i])
    return {
        'MQ0': {
            'errors': error_mq0,
            'OK': ok_mq0
        }
    }

In [12]:
mendelian_stats = get_stats()
fig, axs = plt.subplots(1, 1, squeeze=False, sharex=True, figsize=(16, 9))
ax = axs[0][0]
MQ0_stats = mendelian_stats['MQ0']
ax.set_title('MQ0')
ax.boxplot([MQ0_stats['errors'], MQ0_stats['OK']])
ax.set_ylim(0, 200)
pass

NameError: name 'acceptable_position_to_genotype' is not defined

In [ ]:
#Stats by position

In [ ]:
import numpy as np
a = np.array([[-1, -1]])
np.where(a==-1)